In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import os
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
%matplotlib inline

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# set cwd back to default
os.chdir('../')
os.getcwd()

'/Users/Tristan/Downloads/dog-breed-identification/src'

In [3]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'labels' = pandas.df('img_name','classification')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [4]:
import data # src/data.py
dataset = data.init_dataset()

0       cardboard
1       cardboard
2       cardboard
3       cardboard
4       cardboard
5       cardboard
6       cardboard
7       cardboard
8       cardboard
9       cardboard
10      cardboard
11      cardboard
12      cardboard
13      cardboard
14      cardboard
15      cardboard
16      cardboard
17      cardboard
18      cardboard
19      cardboard
20      cardboard
21      cardboard
22      cardboard
23      cardboard
24      cardboard
25      cardboard
26      cardboard
27      cardboard
28      cardboard
29      cardboard
          ...    
2497        trash
2498        trash
2499        trash
2500        trash
2501        trash
2502        trash
2503        trash
2504        trash
2505        trash
2506        trash
2507        trash
2508        trash
2509        trash
2510        trash
2511        trash
2512        trash
2513        trash
2514        trash
2515        trash
2516        trash
2517        trash
2518        trash
2519        trash
2520        trash
2521      

### Select the amount of classes that will be used

In [5]:
# pick the n classes with the most occuring instances
amt = 6
classes = data.top_classes(dataset.labels, amt)
classes

['paper', 'glass', 'plastic', 'metal', 'cardboard', 'trash']

In [6]:
def extract_topx_classes(classes, train_or_test):
    name_list = []
    n_per_class = []
    tail = '.jpg'
    for cls in classes:
        print(cls)
        names = data.items_with_label(dataset.labels, cls)
        if train_or_test == 'train':
            train_names = [f for f in names if (f) in dataset.train]
        if train_or_test == 'test':
            train_names = [f for f in names if (f) in dataset.test]
        name_list.append(train_names)
        n_per_class.append(len(train_names))

    n = min(n_per_class)
    # (optional) reduce n to check whether the model can rember its input
    reduced_n = 50
    if n > reduced_n:    n = reduced_n
    x = []
    for ls in name_list:
        for name in ls[:n]:
            x.append(name)
    random.shuffle(x)
    return x

x_train = extract_topx_classes(classes, 'train')
x_test = extract_topx_classes(classes, 'test')


paper
glass
plastic
metal
cardboard
trash
paper
glass
plastic
metal
cardboard
trash


### Load and convert images

In [8]:
x_train, y_train, n = data.extract_all(dataset, x_train)
y_test = y_train
y_train, y_test = data.labels_to_vectors(dataset, y_train, y_test)

input_shape = x_train.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label


('extract all data:', 222)


In [7]:
x_test, y_test, n = data.extract_all_test(dataset, x_test)


('extract all data:', 300)


Initiate VG16 network and add extra layers

In [9]:
from keras.models import Model 
dropout = 0.10

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape = input_shape )
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
# top_model = Sequential()
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(1, activation='sigmoid'))

for layer in model.layers:
    layer.trainable = False
    
#Adding custom Layers 
x = model.output
x = Flatten()(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation="relu")(x)
x = (Dense(128, activation='relu'))(x)
x = (Dropout(dropout))(x)
x = (Dense(16, activation='relu'))(x)
predictions = Dense(output_length, activation="softmax")(x)


# creating the final model 
model_final = Model(input = model.input, output = predictions)

learning_rate = 0.001
# Adam, SGD
# sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
optimizer = optimizers.Adam(lr=learning_rate)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy',
    'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])

Model loaded.


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


data augmentation

In [10]:
#Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)


### Train the model

In [11]:
batch_size = 8
# n epochs = n iterations over all the training data
epochs = 10

### With data augmentation

In [12]:
train_datagen.fit(x_train)

model_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])


Epoch 1/10
27/27 [==============================] - 437s 16s/step - loss: 4.7202 - acc: 0.1604 - mean_squared_error: 0.1965 - categorical_crossentropy: 4.7202 - top_k_categorical_accuracy: 0.8272
Epoch 2/10
27/27 [==============================] - 484s 18s/step - loss: 1.8981 - acc: 0.1945 - mean_squared_error: 0.1412 - categorical_crossentropy: 1.8981 - top_k_categorical_accuracy: 0.8272
Epoch 3/10
12/27 [============>.................] - ETA: 6:01 - loss: 1.7735 - acc: 0.1562 - mean_squared_error: 0.1380 - categorical_crossentropy: 1.7735 - top_k_categorical_accuracy: 0.8438

KeyboardInterrupt: 

### Without data augmentation

In [ ]:
from keras.callbacks import TensorBoard
model_final.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

In [ ]:
model_final.summary()

### running tests


In [ ]:
cvscores = []
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))